In [2]:
import os 

os.chdir('E://bond-dex/bond-amm-research/')

In [3]:
import pandas as pd 
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt 

In [4]:
class Rate(): 
    
    def __init__(self, T, N, r0, a, b, sigma): 
        self.T = T
        self.N = N 
        self.r0 = r0 
        self.a = a 
        self.b = b 
        self.sigma = sigma 
        self.dt = T / N
        
        np.random.seed(5)
        
    def cir(self): 
        
        out = np.zeros(self.N)
        
        out[0] = r0 
        
        for i in range(self.N - 1):
            dr_i = self.a * (self.b - out[i]) * self.dt + self.sigma * np.sqrt(out[i]) * np.random.normal() * np.sqrt(self.dt)
            out[i + 1] = out[i] + dr_i
        
        return out

In [5]:
def calculate_x(r, r_min, r_max, A, t, T): 
    tau = T - t 
    return A * (r - r_min) * np.exp(-(r_max - r)**2 / (2 * (r_max - r_min)) * tau)


def calculate_y(r, r_min, r_max, A, t, T): 
    tau = T - t 
    return A * (r_max - r) * np.exp(-(r_max - r)**2 / (2 * (r_max - r_min)) * tau - r * tau)

In [6]:
T = 1
N = int(T * 365 * 24 * 60)

sigma=0.05
a=5
b=0.05

r0 = 0.05

y0 = 0.5

In [7]:
rate = Rate(T=T, N=N, r0=r0, a=a, b=b, sigma=sigma)

r = rate.cir()

In [8]:
r_min, r_max = 0.01, 0.1

In [9]:
A = y0 / calculate_y(r0, r_min, r_max, A=1, t=0, T=T)

In [10]:
x = calculate_x(r, r_min, r_max, A=A, t=np.arange(0, N) * T / N, T=T)
y = calculate_y(r, r_min, r_max, A=A, t=np.arange(0, N) * T / N, T=T)

<font color='red'>**Sample trades over time**</font>

In [11]:
dx = np.diff(x)
dy = np.diff(y)

In [12]:
def sample_trade(dx, dy):
    N = len(dx)
    
    trade_side = np.random.binomial(n=1, p=0.5, size=N)
    
    trade_size = np.where(trade_side == 1, dy, dx)
    
    return trade_side, trade_size

In [13]:
trade_sides, trade_volumes = sample_trade(dx, dy)

### Save simulated trade to csv

In [14]:
trade_df = pd.DataFrame({'dx': dx, 'dy': dy})
trade_df['action'] = trade_df.dx.apply(lambda x: 'lend' if x < 0 else 'borrow')
trade_df = trade_df.reset_index().rename(
    columns={'index': 'timestamp', 'dx': 'bond_volume'}
)[['timestamp', 'action', 'bond_volume']]
trade_df.bond_volume = np.abs(trade_df.bond_volume)
trade_df.timestamp += 1

In [15]:
trade_df.groupby('action').agg({'bond_volume': 'describe'})

bond_volume                                                        \
             count      mean       std           min       25%       50%   
action                                                                     
borrow    262119.0  0.000131  0.000099  2.817472e-10  0.000053  0.000111   
lend      263480.0  0.000131  0.000099  2.215225e-10  0.000052  0.000110   

                            
             75%       max  
action                      
borrow  0.000189  0.000737  
lend    0.000189  0.000746

In [16]:
trade_df

,timestamp,action,bond_volume
0,1,borrow,0.000073
1,2,lend,0.000055
2,3,borrow,0.000403
3,4,lend,0.000042
4,5,borrow,0.000018
...,...,...,...
525594,525595,lend,0.000003
525595,525596,borrow,0.000276
525596,525597,lend,0.000138
525597,525598,lend,0.000058


In [17]:
import enum

In [18]:
class Action(enum.Enum):
    
    OPEN_POSITION = 'open_position'
    LIQUIDATE = 'liquidate'
    
class Actor(enum.Enum): 
    
    LENDER = 'lender'
    BORROWER = 'borrower'

In [19]:
def make_action(df, timestamp_pivot): 
    
    df['position'] = df.actor_signal * df.sign
    
    df['actor'] = df.actor_signal.map({1: Actor.LENDER.value, -1: Actor.BORROWER.value})
    df['action'] = df.position.map({1: Action.OPEN_POSITION.value, -1: Action.LIQUIDATE.value})
    
    df['actor'] = np.where(df.timestamp <= timestamp_pivot, 
                           df.sign.map({1: Actor.LENDER.value, -1: Actor.BORROWER.value}), 
                           df.actor)
    df['action'] = np.where(df.timestamp <= timestamp_pivot, Action.OPEN_POSITION.value, df.action)
    
    return df

In [20]:
def make_new_input(df, initial_position=0.4, scale_time=1):
    
    df = df.copy()
    
    df['sign'] = df.action.map({'lend': 1, 'borrow': -1})
    df['net_volume'] = df.bond_volume * df.sign
    df['cum_position'] = df.sort_values(by='timestamp', ascending=True).groupby('action', as_index=False)['net_volume'].cumsum()
    
    time_pivot = df[(df.action == 'lend') & (df.cum_position < initial_position)].timestamp.max()
    actor = np.random.binomial(1, 0.5, size=df.shape[0]) * 2 - 1
    
    df['actor_signal'] = actor
    
    df = make_action(df, time_pivot)
    
    df.timestamp = df.timestamp *  scale_time
    
    return df[['timestamp', 'actor', 'action', 'bond_volume']]

In [22]:
def create_input_dx(T, N, r0, a, b, sigma, r_min, r_max):
    rate = Rate(T=T, N=N, r0=r0, a=a, b=b, sigma=sigma)
    r = rate.cir()
    A = y0 / calculate_y(r0, r_min, r_max, A=1, t=0, T=T)
    x = calculate_x(r, r_min, r_max, A=A, t=np.arange(0, N) * T / N, T=T)
    y = calculate_y(r, r_min, r_max, A=A, t=np.arange(0, N) * T / N, T=T)
    dx = np.diff(x)
    dy = np.diff(y)
    trade_sides, trade_volumes = sample_trade(dx, dy)
    trade_df = pd.DataFrame({'dx': dx, 'dy': dy})
    trade_df['action'] = trade_df.dx.apply(lambda x: 'lend' if x < 0 else 'borrow')
    trade_df = trade_df.reset_index().rename(
        columns={'index': 'timestamp', 'dx': 'bond_volume'}
    )[['timestamp', 'action', 'bond_volume']]
    trade_df.bond_volume = np.abs(trade_df.bond_volume)
    trade_df.timestamp = trade_df.timestamp * 10 + 1

    dx_df = make_new_input(trade_df, initial_position=0.4, scale_time=1)
    return dx_df

In [41]:
dx_df = create_input_dx(T, N, r0, a, b, 0.5, 0.01, 0.1)

In [42]:
dx_df.bond_volume *= 10

In [43]:
dx_df.to_csv(f'E:/bond-dex/bond-amm-simulation/sources/input/dx/synthetic_calibrate_4.csv', index=False)

In [44]:
dx_df.describe()

,timestamp,bond_volume
count,5.255990e+05,5.255990e+05
mean,2.627991e+06,1.178987e-02
std,1.517275e+06,1.020532e-02
min,1.000000e+00,4.127565e-08
25%,1.313996e+06,4.094170e-03
50%,2.627991e+06,9.065695e-03
75%,3.941986e+06,1.671256e-02
max,5.255981e+06,1.040785e-01


In [52]:
for version, sigma, r_min in zip([1, 2, 3], [0.05, 0.2, 0.5], [0.01, 0.01, 0.001]):

    dx_df = create_input_dx(T, N, r0, a, b, sigma, r_min, r_max)
    
    dx_df.to_csv(f'E:/bond-dex/bond-amm-simulation/sources/input/dx/synthetic_calibrate_{version}.csv', index=False)

In [31]:
dx_df

,timestamp,actor,action,bond_volume
0,1,borrower,open_position,0.002925
1,11,lender,open_position,0.002195
2,21,borrower,open_position,0.016119
3,31,lender,open_position,0.001678
4,41,borrower,open_position,0.000729
...,...,...,...,...
525594,5255941,lender,open_position,0.000062
525595,5255951,borrower,open_position,0.005398
525596,5255961,borrower,liquidate,0.002711
525597,5255971,borrower,liquidate,0.001134
